# Imports

In [1]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import torch
import torch.nn as nn
from numpy import mean
from igraph import Graph as igraphGraph
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial.distance
from scipy.spatial.distance import cdist
from shapely.geometry import LineString, MultiLineString, Polygon, MultiPolygon, LinearRing

# Input

In [2]:
transformation = Transformation()

user_number_triangles = 500   #à diminuer si le process est trop long
number_neigh_tri = 20

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [3]:
if len(graph._node)<20:
    raise Exception("Input mesh does not have enough vertices. (More than 20 is needed)")

# Point Sampler

### DevConv

In [4]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [5]:
class DevConv():
    def __init__(self, graph, output_dimension):
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = np.random.random((output_dimension))      #change
        self.W_theta = np.array([0.1, 0.1, 0.1]).reshape(1, -1)  # change
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))                               #list of "output_dimension" for each "list_node" element
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):       # for each node and its adjacency nodes
            neigh_nodes = np.array(list(dict_neigh.keys()))                                             # array of adjacency nodes
            diff = current_node - neigh_nodes                                                           # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            neigh_distances = np.linalg.norm(self.W_theta * diff, axis=1)                               # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = np.mean(list_inc_score, axis=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        return np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0)
        

In [6]:
devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = sigmoid(inclusion_score)
print(inclusion_score)
print(inclusion_score.shape)

[0.6595211  0.65990481 0.6595211  ... 0.61399193 0.89125932 0.61399193]
[0.66237132 0.66275669 0.66237132 ... 0.61664539 0.89511104 0.61664539]
[0.59132718 0.59137912 0.59132718 ... 0.58514965 0.62228802 0.58514965]
(5999,)


### Multinomial Sampling

In [7]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

[0 0 0 ... 0 0 0]


In [8]:
target_number_point = min(len(graph._node), user_number_triangles*3)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]     # Take the k ("target_number_points") largest values given by the multinomial sampling 
list_k_nodes = np.array(graph)[index_k_nodes]                                # Take the selected nodes (list of list of 3)
list_k_nodes[:5]

array([[89.11374  ,  5.2797017, 88.39131  ],
       [87.81501  ,  5.2797017, 90.51065  ],
       [86.77158  ,  5.2797017, 91.79918  ],
       [90.06495  ,  5.2797017, 86.0949   ],
       [90.30048  ,  5.2797017, 85.29975  ]], dtype=float32)

# KNN extended graph

In [9]:
"""
_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points

extended_graph = nx.Graph()
for index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node
    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates
    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)
        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples
        extended_graph.add_edge(*edge)                                      # Add the edge to the graph

transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)
"""

"\n_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points\n\nextended_graph = nx.Graph()\nfor index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node\n    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates\n    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)\n        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples\n        extended_graph.add_edge(*edge)                                      # Add the edge to the graph\n\ntransformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)\n"

In [10]:
def knn_and_extended_graph(list_k_nodes_i, k_neighbors=15):
    """
    Perform k-NN on the given points and construct an extended graph.

    Parameters:
    - list_k_nodes_i: array-like
        List of coordinates of the selected points.
    - k_neighbors: int, optional (default=15)
        Number of neighbors for k-NN.

    Returns:
    - extended_graph_i: networkx.Graph
        Extended graph.
    """

    # Perform k-NN on the selected points
    _, indices = NearestNeighbors(n_neighbors=k_neighbors).fit(list_k_nodes_i).kneighbors(list_k_nodes_i)

    # Construct the extended graph
    extended_graph_i = nx.Graph()
    for poly in indices:
        current_node = tuple(list_k_nodes_i[poly[0]])
        for index_other_node in poly[1:]:
            edge = current_node, tuple(list_k_nodes_i[index_other_node])
            extended_graph_i.add_edge(*edge)

    return extended_graph_i

extended_graph = knn_and_extended_graph(list_k_nodes)
transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)

Number of nodes: 1500
Number of edges: 11736


# Edge Predictor

In [11]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
inclusion_score.shape

(1500, 64)

In [12]:
class SparseAttentionEdgePredictorLayer(nn.Module):
    def __init__(self, nodes, neighbors, size=64):
        super().__init__()
        self.size = size
        self.nodes = nodes
        self.neighbors = neighbors
        self.wq = nn.Parameter(torch.Tensor(size))
        self.wk = nn.Parameter(torch.Tensor(size))

        nn.init.normal_(self.wq)
        nn.init.normal_(self.wk)

    def forward(self, f):
        wq_f = self.wq.reshape(-1, 1) * f                   # Wq*f
        wk_f = self.wk.reshape(-1, 1) * f                   # Wq*f
        S = torch.exp(torch.matmul(wq_f.T, wk_f))           # e^((wq_f.T)*(wk_f))
        
        nonzero_neigh = self.neighbors.nonzero()                                                    # Find indexes of neighbors in graph
        unique_first_elements, counts = torch.unique(nonzero_neigh[:, 0], return_counts=True)       # Count number of neighbors per node
        split_tensors = list(torch.split(nonzero_neigh, tuple(counts)))                             # split indexes of neighbors into a list (1 element = 1 tensor of indexes)

        temp = [[S[n[i,0], n[i,1]] for i in range(len(n))] for n in split_tensors]                  # For each node, get the S value for the neighbors indexes
        summed = torch.Tensor([torch.sum(torch.Tensor(e)) for e in temp])                           # Sum these results for each nodes
        division = summed.unsqueeze(0).repeat(1, S.shape[1], 1)[0]                                  # Repeat the sum in S.shape[1] array => division per columns
        final_term  = S / division

        return final_term


f = np.mean(inclusion_score, axis=1)                            # Flatten the matrix of inclusion score 
layer = SparseAttentionEdgePredictorLayer(torch.Tensor(list(extended_graph.nodes())), torch.Tensor(nx.adjacency_matrix(extended_graph).toarray()))
S = layer.forward(torch.Tensor(f))
print(S.shape)

torch.Size([1500, 1500])


### Sparse Attention

In [13]:
# S = S*np.random.choice([0, 1], size=S.shape)      # Add a random mask to emulate the 'sparse'

# Face Candidates

#### Inputs

In [14]:
adjacency = nx.adjacency_matrix(extended_graph)
# S = np.random.rand(target_number_point, target_number_point)
print(adjacency)
print(S)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (1, 5)	1
  (1, 7)	1
  (1, 8)	1
  (1, 11)	1
  (1, 12)	1
  (1, 13)	1
  (1, 14)	1
  :	:
  (1498, 1463)	1
  (1498, 1464)	1
  (1498, 1465)	1
  (1498, 1475)	1
  (1498, 1491)	1
  (1498, 1492)	1
  (1498, 1493)	1
  (1498, 1494)	1
  (1498, 1495)	1
  (1498, 1496)	1
  (1498, 1499)	1
  (1499, 1462)	1
  (1499, 1463)	1
  (1499, 1464)	1
  (1499, 1465)	1
  (1499, 1466)	1
  (1499, 1483)	1
  (1499, 1489)	1
  (1499, 1491)	1
  (1499, 1492)	1
  (1499, 1493)	1
  (1499, 1494)	1
  (1499, 1495)	1
  (1499, 1496)	1
  (1499, 1498)	1
tensor([[0.0714, 0.0628, 0.0669,  ..., 0.0649, 0.0717, 0.0718],
        [0.0714, 0.0628, 0.0669,  ..., 0.0649, 0.0717, 0.0718],
        [0.0718, 0.0632, 0.0672,  ..., 0.0654, 0.0722, 0.0723],
        ...,
        [0.0701, 0.0617, 0.0658,  ..., 0.0632, 0.0704, 0.0702],
        [0.0713,

In [15]:
A_s = np.zeros((target_number_point,target_number_point))   # Init A_s to 0
A_s = np.matmul(np.matmul(S, adjacency.toarray()), S.T)     # A_s = S * A * S.T
A_s = A_s/A_s.max()                                         # Normalize

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
print(A_s)  # symmétrique
print(A_s.shape)

[[3.77317640e-32 3.78960629e-32 3.86623573e-32 ... 5.82539176e-31
  8.44776855e-31 8.26253392e-31]
 [3.78960629e-32 3.80608375e-32 3.89744609e-32 ... 5.84326241e-31
  8.44162180e-31 8.26461006e-31]
 [3.86623573e-32 3.89744609e-32 3.99135018e-32 ... 5.98038937e-31
  8.68159401e-31 8.48203340e-31]
 ...
 [5.82539176e-31 5.84326241e-31 5.98038937e-31 ... 1.19301961e-29
  1.55922548e-29 1.49034458e-29]
 [8.44776855e-31 8.44162180e-31 8.68159401e-31 ... 1.55922548e-29
  2.55336088e-29 2.35371110e-29]
 [8.26253392e-31 8.26461006e-31 8.48203340e-31 ... 1.49034458e-29
  2.35371110e-29 2.19917201e-29]]
(1500, 1500)


# Face Classifier

### TriConv

#### Inputs

In [ ]:
igraph_g = igraphGraph(directed=extended_graph.is_directed()).from_networkx(extended_graph)
print(igraph_g.summary())

IGRAPH U--- 1500 11835 -- 
+ attr: _nx_name (v)


In [ ]:
triangles_ids_igraph = np.array(igraph_g.cliques(min=3, max=3))
print(triangles_ids_igraph[:3])
print(len(triangles_ids_igraph))

[[682 685 687]
 [764 882 884]
 [882 884 942]]
32790


In [ ]:
triangles = np.array(igraph_g.vs['_nx_name'])[triangles_ids_igraph]   #les triangles
# triangles = [list(map(tuple, row)) for row in triangles.tolist()]
print(triangles[:3])
print(len(triangles))

[[[75.07823    -1.3272538  81.26335   ]
  [75.18281    -1.9813722  81.33282   ]
  [75.142204   -1.9813722  81.3756    ]]

 [[75.01782    -0.66553295  3.7821789 ]
  [75.31144    -3.2518802   3.2605693 ]
  [75.53943    -3.2518802   3.4885595 ]]

 [[75.31144    -3.2518802   3.2605693 ]
  [75.53943    -3.2518802   3.4885595 ]
  [75.60927    -3.2518802   3.6704957 ]]]
32790


In [ ]:
# Extract indices for each triangle
i, j, k = triangles_ids_igraph.T

# Extract probabilities using advanced indexing
A_s_ij = A_s[i, j]
A_s_ik = A_s[i, k]
A_s_jk = A_s[j, k]

# Calculate the barycenter probabilities
p_init = np.zeros(len(triangles))
p_init = (A_s_ij + A_s_ik + A_s_jk) / 3

print(p_init.shape)
print(p_init)

(32790,)
[3.66599607e-26 3.92631320e-26 9.07928958e-26 ... 9.85191834e-14
 9.74778633e-14 8.43601564e-14]


#### Calculate barycenter

In [ ]:
barycenters = np.mean(triangles, axis=1)
print(barycenters.shape)
print(barycenters[:3])

(32790, 3)
[[75.13442   -1.7633328 81.32392  ]
 [75.28956   -2.3897645  3.510436 ]
 [75.48671   -3.25188    3.4732082]]


#### KNN Tri

In [ ]:
_, indices_neigh_tri = NearestNeighbors(n_neighbors=number_neigh_tri).fit(barycenters).kneighbors(barycenters)          # Find k='number_neigh_tri' neighbors of each triangles based on theirs barycenters

Diff Barycenters

In [ ]:
# Calculates the difference between each barycenters and their n='number_neigh_tri'-1 neighbors
barycenters_diff = np.subtract(barycenters[indices_neigh_tri[:, 0]][:, np.newaxis], barycenters[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire

print(barycenters_diff.shape)

(32790, 19, 3)


#### calculate e norm matrix

In [ ]:
diff_vectors = list()

for triangle in triangles:
    e_ij = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[1]))    #Calculate the vectors for each edge of each triangles
    e_ik = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[2]))
    e_jk = np.linalg.norm(np.array(triangle[1]) - np.array(triangle[2]))
    diff_vectors.append([e_ij, e_ik, e_jk])

diff_vectors = np.array(diff_vectors)
print(diff_vectors.shape)
print(diff_vectors[:2])

(32790, 3)
[[0.66605747 0.66675615 0.05898531]
 [2.6547089  2.6547084  0.32242608]]


#### Calculate r

In [ ]:
max_diff_vectors = diff_vectors.max(axis=1)       # calculate t_n_max
min_diff_vectors = diff_vectors.min(axis=1)       # calculate t_n_min
max_diff_vectors.shape

(32790,)

In [ ]:
max_diff_vectors_diff = np.subtract(max_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], max_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_max - t_m_max
print(max_diff_vectors_diff.shape)
min_diff_vectors_diff = np.subtract(min_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], min_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_min - t_m_min
print(min_diff_vectors_diff.shape)

(32790, 19)
(32790, 19)


In [ ]:
r_matrix = np.zeros((len(triangles), number_neigh_tri-1, 5))

r_matrix[:, :, 0]   = min_diff_vectors_diff
r_matrix[:, :, 1]   = max_diff_vectors_diff
r_matrix[:, :, 2:5] = barycenters_diff

print(r_matrix.shape)  # nb_triangles, len(indices_neigh_tri), 5dim/triangles

# Here r_matrix[i,j] represent the relation between the triangles (n = i) and (m = indices_neigh_tri_array[i,j])

(32790, 19, 5)


#### Calculate f

In [ ]:
# Multi Layer Perceptron (MLP) * 3 
f_final = p_init    # TODO


final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

0.9999999999999146
(32790,)
[3.04963754e-05 3.04963754e-05 3.04963754e-05 ... 3.04963754e-05
 3.04963754e-05 3.04963754e-05]


C:\Users\Arthur\AppData\Local\Temp\ipykernel_17108\474768578.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles


# Simplified Mesh

In [ ]:
selected_triangles_indexes = np.argpartition(final_scores, -int(target_number_point/3))[-int(target_number_point/3):] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)
print(selected_triangles[0])

(500, 3, 3)
[[ 51.         -3.0965528 -13.237194 ]
 [ 39.         -8.699333   -3.6727462]
 [ 51.          4.590885    2.507581 ]]


In [ ]:
simplified_final_graph = nx.Graph()
for index_poly, poly in enumerate(selected_triangles):
    for index_current_node in range(len(poly)):
        current_node = tuple(poly[index_current_node])
        for index_other_node in range(index_current_node+1, len(poly)):
            edge = current_node, tuple(poly[index_other_node])
            simplified_final_graph.add_edge(*edge)
            # if attribute do not exists
            if len(simplified_final_graph.nodes[current_node])==0:
                simplified_final_graph.nodes[current_node]['index_triangle'] = set()
            simplified_final_graph.nodes[current_node]['index_triangle'].add(index_poly)
            if len(simplified_final_graph.nodes[tuple(poly[index_other_node])])==0:
                simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
            simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
            
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 43
Number of edges: 203


In [ ]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

#Affichage
transformation.mesh_to_display_vtk(mesh_data)
transformation.mesh_to_display_vtk(simplified_final_mesh)

# Fonctions Loss

## Probabilistic Chamfer distance

In [ ]:
def torch_d_P_Ps(p_y, x, y):
    """All Tensors in input"""
    # print(p_y.shape, x.shape, y.shape)

    expanded_x1 = x.unsqueeze(1)
    expanded_x2 = y.unsqueeze(0)
    distances = torch.norm(expanded_x1 - expanded_x2, dim=2)        # distance matrix

    min_x = distances.min(dim=1).values
    min_y = distances.min(dim=0)

    first_term = torch.sum(torch.index_select(p_y, 0, min_y.indices) * min_y.values)
    second_term = torch.sum(min_x * p_y)

    return first_term + second_term


d_P_Ps = torch_d_P_Ps(torch.Tensor(normalized_inclusion_score), torch.Tensor(np.array(graph)), torch.Tensor(np.array(extended_graph)))

## Probabilistic Surfaces Distance

### d_f_S_Ss

In [ ]:
def torch_d_f_S_Ss(p_b_hat, b_hat, b):
    # print(p_b_hat.shape, b_hat.shape, b.shape)

    expanded_x1 = b_hat.unsqueeze(1)
    expanded_x2 = b.unsqueeze(0)
    distances = torch.norm(expanded_x1 - expanded_x2, dim=2)

    min_b = distances.min(dim=1).values

    final_term = torch.sum(p_b_hat * min_b)

    return final_term



igraph_g_original = igraphGraph(directed=extended_graph.is_directed()).from_networkx(graph)
triangles_ids_igraph_original = np.array(igraph_g_original.cliques(min=3, max=3))
triangles = np.array(igraph_g_original.vs['_nx_name'])[triangles_ids_igraph_original]
torch_b = torch.Tensor(np.mean(triangles, axis=1))

torch_b_hat = torch.Tensor(np.mean(selected_triangles, axis=1))

torch_p_b_hat = torch.Tensor(final_scores[selected_triangles_indexes])

d_f_S_Ss = torch_d_f_S_Ss(torch_p_b_hat, torch_b_hat, torch_b)

### d_r_S_Ss

In [ ]:
def torch_d_r_S_Ss(p_x, p_y, x ,y):
    expanded_x = x.unsqueeze(1)
    expanded_y = y.unsqueeze(0)
    distances = torch.norm(expanded_x - expanded_y, dim=2)
    min_d = distances.min(dim=0).values
    first_term = p_y * min_d


    indices_knn = distances.topk(k=50, dim=0, largest=False).indices.T  # Indices of the k-nearest neighbors
    knn_labels = x[indices_knn]
    xtk = torch.reshape(knn_labels, shape=((y.shape[0])*50, 3))

    expanded_xtk = xtk.unsqueeze(1)
    distances_knn = torch.norm(expanded_xtk - expanded_y, dim=2)
    min_knn = distances_knn.min(dim=1).values
    min_knn_reshaped = min_knn.reshape(((y.shape[0]), 50))

    ptk_time_norm = p_x[indices_knn] * min_knn_reshaped
    factor = (1-p_y) * (1/50)
    second_term = factor * torch.sum(ptk_time_norm, dim=1)

    final_term = torch.sum(first_term + second_term)

    return final_term

d_f_S_Ss = torch_d_r_S_Ss(torch.Tensor(final_scores), torch_p_b_hat, torch_b, torch_b_hat)
d_f_S_Ss

tensor(0.0768)

## Triangle Collision Loss

In [ ]:
def compute_lc_le_lo(p_t, m_c_e_o, Fs):
    """
    Compute the collision loss term L_c.

    Parameters:
    - p_t: 1D numpy array containing the probabilities of each triangle (indices)
    - m_c_t: 2D numpy array containing the number of faces penetrated by each triangle
    - Fs: 3D numpy array representing the vertices of triangles

    Returns:
    - L_c: Collision loss term
    """
    assert len(p_t) == len(m_c_e_o), "Input arrays must have the same length"

    penalty_per_triangle = p_t * m_c_e_o

    # Sum the penalties for all selected triangles
    total_penalty = np.sum(penalty_per_triangle)

    # Compute the collision loss term L_c
    L_c_e_o = (1 / len(Fs)) * total_penalty

    return L_c_e_o

# Example usage:
# Replace the arrays below with your actual data
# p_t = selected_triangles_indexes
# m_c_t = numpy array containing the number of faces penetrated by each triangle
# Fs = 3D numpy array representing the vertices of triangles
p_t = selected_triangles_indexes
Fs = triangles  # Given data

In [ ]:
number_neigh_barycenters = min(50, len(b_hat))
_, indexes_neigh_selected_barycenters = NearestNeighbors(n_neighbors=number_neigh_barycenters).fit(b_hat).kneighbors(b_hat)

NameError: name 'b_hat' is not defined

### Lc

In [ ]:
mc = np.zeros((500))

for index_neigh_barycenters in indexes_neigh_selected_barycenters:
    current_triangle = selected_triangles[index_neigh_barycenters[0]]
    others_triangles = selected_triangles[index_neigh_barycenters[1:]]

    lines_current_triangle = LinearRing(current_triangle)
    polygons_others_tri = MultiPolygon([Polygon(others_triangle) for others_triangle in others_triangles]).buffer(0)    # buffer 0 to correct invalid polygons => take the exterior of the shape


    intersection = lines_current_triangle.intersection(polygons_others_tri)
    if intersection.is_empty:
        continue
    if intersection.geom_type == 'MultiLineString':
        mc[index_neigh_barycenters[0]] = len(intersection.geoms)
    else:
        mc[index_neigh_barycenters[0]] += 1
L_c = compute_lc_le_lo(p_t, mc, Fs)
print("L_c : ", L_c)

L_c :  4225.18776054694


### Le

In [ ]:
me = np.zeros((500))

for index_neigh_barycenters in indexes_neigh_selected_barycenters:
    current_triangle = selected_triangles[index_neigh_barycenters[0]]
    others_triangles = selected_triangles[index_neigh_barycenters[1:]]

    lines_current_triangle = LinearRing(current_triangle)
    lines_others_tri = MultiLineString([LineString([other_tri[0], other_tri[1], other_tri[2], other_tri[0]]) for other_tri in others_triangles])

    intersection = lines_current_triangle.intersection(lines_others_tri)
    if intersection.is_empty:
        continue
    if intersection.geom_type == 'MultiLineString':
        me[index_neigh_barycenters[0]] = len(intersection.geoms)
    else:
        me[index_neigh_barycenters[0]] += 1
L_e = compute_lc_le_lo(p_t, me, Fs)
print("L_e : ", L_e)

L_e :  24105.658829414708


### Lo

In [ ]:
mo = np.zeros((500))

Lo - échantillonnage de 100 points à partir de chaque triangle

In [ ]:

def sample_points_from_triangle(t, num_points=100):
    v1, v2, v3 = t
    bary_coords = np.random.rand(num_points, 2)
    sqrt_bary_coords = np.sqrt(bary_coords[:, 0])

    u = sqrt_bary_coords
    v = bary_coords[:, 1]

    """
    La formule spécifique est dérivée de l'expression générale d'interpolation barycentrique 
    sommets A, B et C
    coord barycentriques: u et v
    coord cartésiennes: x,y,z 
    """
    x_coords = (1 - u - v) * v1[0] + u * v2[0] + v * v3[0]
    y_coords = (1 - u - v) * v1[1] + u * v2[1] + v * v3[1]
    z_coords = (1 - u - v) * v1[2] + u * v2[2] + v * v3[2]

    sampled_points = np.column_stack((x_coords, y_coords, z_coords))
    return sampled_points

points100 = sample_points_from_triangle(triangle, num_points=100)
points100
points100.shape

(100, 3)

Les Aires

In [ ]:
number_neigh_selected_barycenters = min(50, len(b_hat))
def knnbar(nn):
  _, indexes_neigh_selected_barycenters = NearestNeighbors(n_neighbors=nn).fit(b_hat).kneighbors(b_hat)
  return _, indexes_neigh_selected_barycenters

In [ ]:
d, indices = knnbar(number_neigh_selected_barycenters)
indices

array([[  0, 292,  20, ..., 181, 180, 141],
       [  1,   6, 344, ..., 397, 151, 473],
       [  2, 186,  10, ..., 185, 184, 189],
       ...,
       [497, 421, 418, ..., 379, 238, 457],
       [498, 261, 303, ..., 385,  40, 327],
       [499, 264, 126, ..., 404, 338, 320]], dtype=int64)

In [ ]:
from scipy.spatial import distance as scipy_distance

def calculate_triangle_area(triangle):
    # Fonction pour calculer l'aire d'un triangle en utilisant la formule de Héron
    side_lengths = [scipy_distance.euclidean(triangle[i], triangle[(i + 1) % 3]) for i in range(3)]
    s = sum(side_lengths) / 2
    return np.sqrt(s * np.prod([s - length for length in side_lengths]))

def penalize_overlapping_triangles(points_list, triangles, k=50):
    # Fonction pour pénaliser les triangles qui se chevauchent
    assignment_results = []
    overlapping_triangles = np.zeros(len(triangles), dtype=int)  # Déclaration en dehors de la boucle

    # ajustement de la valeur maximale de k en fonction du nombre de triangles
    k = min(k, len(triangles))

    # NearestNeighbors pour trouver les k triangles les plus proches pour chaque point
    knn_model = NearestNeighbors(n_neighbors=k).fit(np.vstack(triangles))
    d, indices = knn_model.kneighbors(np.array(points_list)[:, 0, :])

    for i, point_list in enumerate(points_list):
        point = point_list[0]

        # Verif que les indices sont valides
        closest_triangle_indices = indices[i, :k]
        closest_triangle_indices = closest_triangle_indices[closest_triangle_indices < len(triangles)]

        # Accéder aux triangles en utilisant les indices valides
        valid_triangle_indices = []
        for idx in closest_triangle_indices:
            modified_triangle = np.copy(triangles[idx])
            modified_triangle[0] = point

            area_original = calculate_triangle_area(triangles[idx])
            area_modified = calculate_triangle_area(modified_triangle)

            # Vérifier si la somme des aires est proche de l'aire du triangle
            if not np.isclose(area_original, area_modified, rtol=1e-5):
                assignment_results.append((point, triangles[idx]))
                valid_triangle_indices.append(idx)

        # MAJ du overlapping_triangles avec les indices valides
        overlapping_triangles[valid_triangle_indices] += 1

    penalties = overlapping_triangles
    total_penalty = np.sum(penalties)
    Lo = (1 / len(triangles)) * total_penalty

    return Lo


L_o = penalize_overlapping_triangles([points100], triangles, k=50)
print("Pénalité pour les triangles qui se chevauchent :", L_o)


Pénalité pour les triangles qui se chevauchent : 0.0008337502084375521


## Final Loss

lambda_c, lambda_e, lambda_o = 0.01, 0.01, 0.01


Loss_L = prob_chamfer_dist + d_f_S_Ss + d_r_S_Ss + (lambda_c * L_c) + (lambda_e * L_e) + (lambda_o * L_o)
print(Loss_L)